# Classification of movements using convnets

**Basic Imports**

In [18]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from scipy.io import loadmat
%matplotlib inline

In [2]:
def load_data_DB1(subject,exercise):
    data = loadmat(f'../Dataset/DB1/s{subject}/S{subject}_A1_E{exercise}.mat')
    return data

In [3]:
def get_start_indexes(data,movement,target):
    start_indexes=[]
    for idx,move in enumerate(data[target]):
        if move[0] == movement and data[target][idx-1][0] == 0:
            start_indexes.append(idx)
        if len(start_indexes) == 10:
            break
    return start_indexes

In [4]:
def get_movement_df(data,movement,target):
    '''
    Returns the repetitions of a movement based on fixed length of repetition.
    if target is stimulus then each rep takes 5 seconds but if target is restimulus then each rep takes 2 seconds
    '''
    start_indexes = get_start_indexes(data,movement,target)
    df= pd.DataFrame()
    step = 500 if target == 'stimulus' else 200
    for i in start_indexes:
        new_row = data['emg'][i:i+step]
        df = df.append(pd.DataFrame(new_row),ignore_index=True)
    target_movement = np.full((2000,1),movement,dtype=int)
    df['target'] = target_movement
    return df

## Classification of exercise 1 movements for one subject

In [5]:
data = load_data_DB1(subject=10, exercise=1)
ex1_moves = np.arange(1,13)
data_df = pd.DataFrame()
for m in ex1_moves:
    move_df = get_movement_df(data=data, movement=m, target='restimulus')
    data_df = data_df.append(move_df, ignore_index=True)
    
data_df

,0,1,2,3,4,5,6,7,8,9,target
0,0.0024,0.0024,0.0146,0.0049,0.0024,0.0024,0.0439,0.0757,0.0073,0.0244,1
1,0.0024,0.0024,0.0244,0.0073,0.0024,0.0024,0.0513,0.0879,0.0098,0.0269,1
2,0.0024,0.0024,0.0244,0.0049,0.0049,0.0024,0.0488,0.1001,0.0098,0.0244,1
3,0.0024,0.0024,0.0171,0.0024,0.0024,0.0024,0.0464,0.1025,0.0024,0.0220,1
4,0.0024,0.0024,0.0122,0.0024,0.0049,0.0024,0.0513,0.0977,0.0024,0.0220,1
...,...,...,...,...,...,...,...,...,...,...,...
23995,0.0024,0.2197,0.3613,0.0757,0.0244,0.0391,0.2344,1.3696,1.2085,0.5933,12
23996,0.0024,0.2124,0.3760,0.0757,0.0220,0.0464,0.2441,1.4038,1.2769,0.5640,12
23997,0.0024,0.2319,0.3735,0.0781,0.0244,0.0537,0.2808,1.5430,1.2622,0.5493,12
23998,0.0024,0.2368,0.3516,0.0781,0.0244,0.0610,0.2979,1.5430,1.2329,0.6372,12


In [6]:
import math
X = np.zeros(shape=(120,200,10))
y = np.zeros(shape=(120,1), dtype='int32')
move_label = 1
for i in np.arange(0,120):
    start = i*200
    end = start + 200
    move = data_df.iloc[start:end,:-1]
    X[i] = move
    y[i] = math.floor(i/10) + 1

In [13]:
X.shape, y.shape

((120, 200, 10), (120, 1))

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((96, 200, 10), (24, 200, 10), (96, 1), (24, 1))

In [19]:
X_train = X_train.reshape((96, 200,10,1))
X_test = X_test.reshape((24,200,10,1))

In [16]:
# scaler = MinMaxScaler()
# X_train_transformed = scaler.fit_transform(X_train)
# X_test_transformed = scaler.fit_transform(X_test)
X_train_transformed = X_train_transformed.reshape((96, 200,10,1))
X_test_transformed = X_test_transformed.reshape((24,200,10,1))

ValueError: Found array with dim 3. MinMaxScaler expected <= 2.

In [40]:
from keras.utils import to_categorical

y_train_transformed = to_categorical(y_train, dtype='int32')
y_test_transformed = to_categorical(y_test, dtype='int32')

In [47]:
y_train_transformed = np.delete(y_train_transformed, 0, axis=1)
y_test_transformed = np.delete(y_test_transformed, 0, axis=1)

## Preparing the model

In [78]:
from keras import layers, models

model = models.Sequential()
model.add(layer=layers.Conv2D(filters=32 , kernel_size=(3,3), activation='relu',padding='same', input_shape=(200,10,1)))
model.add(layer=layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu',padding='same'))
model.add(layer=layers.MaxPooling2D(pool_size=(2,2)))
model.add(layer=layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu',padding='same'))
model.add(layer=layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu',padding='same'))
model.add(layer=layers.MaxPooling2D(pool_size=(2,2)))
model.add(layer=layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu',padding='same'))
model.add(layer=layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu',padding='same'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(12, activation='softmax'))

In [79]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 200, 10, 32)       320       
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 200, 10, 32)       9248      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 100, 5, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 100, 5, 64)        18496     
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 100, 5, 64)        36928     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 50, 2, 64)         0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 50, 2, 64)        

In [83]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train_transformed, epochs=20, batch_size=32)

Epoch 1/20
3/3 [==============================] - 1s 173ms/step - loss: 1.0067 - accuracy: 0.8021
Epoch 2/20
3/3 [==============================] - 0s 149ms/step - loss: 1.2264 - accuracy: 0.6771
Epoch 3/20
3/3 [==============================] - 0s 143ms/step - loss: 0.5051 - accuracy: 0.8438
Epoch 4/20
3/3 [==============================] - 0s 138ms/step - loss: 0.4189 - accuracy: 0.8958
Epoch 5/20
3/3 [==============================] - 0s 142ms/step - loss: 0.4032 - accuracy: 0.9062
Epoch 6/20
3/3 [==============================] - 0s 139ms/step - loss: 0.3412 - accuracy: 0.9271
Epoch 7/20
3/3 [==============================] - 0s 132ms/step - loss: 0.2531 - accuracy: 0.9792
Epoch 8/20
3/3 [==============================] - 0s 140ms/step - loss: 0.1579 - accuracy: 1.0000
Epoch 9/20
3/3 [==============================] - 0s 133ms/step - loss: 0.0846 - accuracy: 0.9896
Epoch 10/20
3/3 [==============================] - 0s 138ms/step - loss: 0.0461 - accuracy: 1.0000
Epoch 11/20
3/3 [==

In [84]:
test_loss, test_acc = model.evaluate(X_test, y_test_transformed)

1/1 [==============================] - 0s 5ms/step - loss: 0.1191 - accuracy: 0.9583
